In [ ]:
#Check if colab is running
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    %tensorflow_version 2.x
    
#import TF  
import tensorflow as tf
from platform import python_version
print("Tensorflow version", tf.__version__)
print("Python version =",python_version())

Tensorflow version 2.5.0
Python version = 3.7.10


In [ ]:
import os
import numpy as np
np.random.seed(1338)

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams.update({
    "figure.constrained_layout.use" : True,
    "font.size" : 14,
    "figure.figsize" : (7, 5)
})

from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
path = "drive/MyDrive/ML_Projekt/ML_Flowers"
flowers = ["ButterflyBush", "GrapeHyacinth", "BalloonFlower", "BleedingHeart", "FrangipaniFlower", "Daisy", "Black-eyedSusan", "BlanketFlower", "Waterlilies"]

# Inspect Dataset

In [ ]:
shapes = []
images = []
for flower in flowers:
    for entry in os.scandir(path + f"/Original/{flower}/"):
        im = np.array(Image.open(entry.path))
        images.append(im)
        shapes.append(im.shape[:2])

In [ ]:
shapes = np.array(shapes)
ratio = shapes[:,0] / shapes[:,1]

fig, axs = plt.subplots(1, 4, figsize=(22, 5))
axs[0].scatter(shapes[:,1], shapes[:,0], marker="x")
axs[0].set(
    xlabel = "width",
    ylabel = "height",
    aspect = 1
)
for ax, value, label in zip(axs[1:], 
                            [shapes[:,0], shapes[:,1], ratio],
                            ["height", "width", "height / width"]):
    ax.hist(value, bins=50, histtype="stepfilled")
    ax.set(
        xlabel = label,
        ylabel = "#images",
        aspect = "auto"
    )

fig.show()

In [ ]:
# same as above using seaborn
import pandas as pd
import seaborn as sns

df_shapes = pd.DataFrame({
    "height" : shapes[:,0],
    "width" : shapes[:,1]
})

sns.jointplot(df_shapes.height, df_shapes.width, kind="scatter", height=7, ratio=3)
fig, ax = plt.subplots()
ax.hist(ratio, bins=50, histtype="stepfilled")
ax.set(
    xlabel = "height / width",
    ylabel = "#images",
)
fig.show()

### Define maximum Dimensions of images to use

In [ ]:
lower = 60/100
upper = 1/lower
mask = np.logical_and(ratio > lower, ratio < upper)
print(f"Von {len(shapes)} Bildern haben {len(shapes[mask])} ein Seitenverhältnis zwischen {lower} und {upper}")

In [ ]:
print("Die Indizes einiger sehr breiter Bilder:", np.where(ratio < lower))
print("Die Indizes einiger sehr hoher Bilder:", np.where(ratio > upper))

In [ ]:
plt.imshow(images[144]);

In [ ]:
# How to get size of image from a PIL Image object
testpath = f"drive/MyDrive/ML_Projekt/ML_Flowers/Original/{flowers[0]}"

for i, entry in enumerate(os.scandir(testpath)):
    if i == 129:
        im = Image.open(entry.path)
        print(im.getbbox()) #(left, upper, right, lower)

# Resize pictures

In [ ]:
### Resizen von Bildern auf 200x200
img_rows = 200
img_cols = 200
shape_ord = (img_rows, img_cols, 3)

lower = 60/100
upper = 1/lower

if not os.path.isdir(path + "/Resized/"):
    os.mkdir(path + "/Resized/")

for flower in flowers:
    if not os.path.isdir(path + "/Resized/" + flower):
        folder = path + "/Resized/" + flower
        os.mkdir(folder)
        for entry in os.scandir(path + f"/Original/{flower}/"):
            im = Image.open(entry.path)
            # only use images with: lower < ratio < upper
            ratio = im.getbbox()[3] / im.getbbox()[2] # height / width
            if ratio > lower and ratio < upper:
                im_r = im.resize((img_rows, img_cols), Image.ANTIALIAS)
                im_r.save(path + f"/Resized/{flower}/" + entry.path.split("/")[-1])

# Load and normalize data

In [ ]:
from tensorflow.keras.utils import to_categorical

newpath = path + "/Resized/"

X = []
y = []
for i, flower in enumerate(flowers):
    for entry in os.scandir(newpath + flower):
        X.append(np.array(Image.open(entry.path)))
        y.append(i)

X = np.array(X)
y = np.array(y)

num_classes = len(np.unique(y))
y = to_categorical(y, num_classes)

In [ ]:
print(X.shape, y[0])
print("Für die einzelnen Klassen sind so viele Instanzen im Datensatz:\n", 
      flowers, "\n", np.sum(y, axis=0))

(3281, 200, 200, 3) [1. 0. 0. 0. 0. 0. 0. 0. 0.]
Für die einzelnen Klassen sind so viele Instanzen im Datensatz:
 ['ButterflyBush', 'GrapeHyacinth', 'BalloonFlower', 'BleedingHeart', 'FrangipaniFlower', 'Daisy', 'Black-eyedSusan', 'BlanketFlower', 'Waterlilies'] 
 [409. 368. 435. 411. 318. 282. 407. 326. 325.]


### Split Test/ Train

In [ ]:
from sklearn.model_selection import train_test_split

X_Train, X_test, y_Train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y) # 15% test set

### Normalization

In [ ]:
X_Train = X_Train.reshape(X_Train.shape[0], img_rows * img_cols * 3)
X_test = X_test.reshape(X_test.shape[0], img_rows * img_cols * 3)
X_Train.shape

(2788, 120000)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X_Train = X_Train.astype('float32')
X_test = X_test.astype('float32')

scaler = MinMaxScaler(feature_range = (0, 1))

X_Train = scaler.fit_transform(X_Train)
X_test = scaler.transform(X_test)

X_Train.shape

(2788, 120000)

In [ ]:
X_Train = X_Train.reshape(X_Train.shape[0],img_rows,img_cols, 3)
X_test = X_test.reshape(X_test.shape[0],img_rows,img_cols, 3)
X_Train.shape

(2788, 200, 200, 3)

# Plotting and evaluate function

In [ ]:
# Plotting functions adapted from exercise 5

def plot_history(network_history):
    fig, [ax1, ax2] = plt.subplots(ncols=2, figsize=(15, 5))
    ax1.set(
      xlabel = "Epochs",
      ylabel = "Loss" 
    )
    ax1.plot(network_history.history['loss'], label="Training")
    ax1.plot(network_history.history['val_loss'], label="Validation")
    ax1.legend()
    ax1.grid()

    ax2.set(
      xlabel = "Epochs",
      ylabel = "Accuracy" 
    )
    ax2.plot(network_history.history['accuracy'], label="Training")
    ax2.plot(network_history.history['val_accuracy'], label="Validation")
    ax2.legend()
    ax2.grid()
    fig.show()
    

import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()


def display_errors(errors_index, img_errors, pred_errors, obs_errors, img_rows, img_cols):
    """ This function shows 6 images with their predicted and real labels """
    nrows = 2
    ncols = 3
    fig, axs = plt.subplots(nrows, ncols, sharex=True, sharey=True, figsize=(10, 8))
    for i, ax in enumerate(axs.flat):
      error = errors_index[i]
      ax.imshow(img_errors[error].reshape(img_rows, img_cols, 3),
                interpolation='nearest')
      ax.set_title(f"Predicted label :{pred_errors[error]}\nTrue label :{obs_errors[error]}")
    
    fig.show()


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def evaluate(X_test, Y_test, label):
    label_name = flowers[label]

    ### Evaluate loss and metrics
    loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
    print('Test Loss:', loss)
    print('Test Accuracy:', accuracy)
    # Predict the values from the test dataset
    Y_pred = model.predict(X_test)
    # Convert one hot vectors to classes 
    Y_cls = np.argmax(Y_pred, axis = 1)
    Y_true = np.argmax(Y_test, axis = 1) 
    print('Classification Report:\n', classification_report(Y_true, Y_cls))
    
    ### Plot 0 probability
    Y_pred_prob = Y_pred[:, label]
    plt.figure(figsize=(8, 5))
    plt.hist(Y_pred_prob[Y_true == label], alpha=0.5, color='red', 
             bins=10, log=True, label=f"True label is {label_name}")
    plt.hist(Y_pred_prob[Y_true != label], alpha=0.5, color='blue',
             bins=10, log=True, label=f"True label is not {label_name}")
    plt.legend()
    plt.xlabel(f'Probability of Label {label_name}')
    plt.ylabel('Number of entries')
    plt.show()
    
    ### compute and plot the confusion matrix
    confusion_mtx = confusion_matrix(Y_true, Y_cls) 
    plot_confusion_matrix(confusion_mtx, classes = flowers)

    ### Plot largest errors
    errors = (Y_cls - Y_true != 0)
    Y_cls_errors = Y_cls[errors]
    Y_pred_errors = Y_pred[errors]
    Y_true_errors = Y_true[errors]
    X_test_errors = X_test[errors]

    # Probabilities of the wrong predicted labels
    Y_pred_errors_prob = np.max(Y_pred_errors, axis = 1)
    # Predicted probabilities of the true values in the error set
    true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))
    
    # Difference between the probability of the predicted label and the true label
    delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors
    sorted_dela_errors = np.argsort(delta_pred_true_errors)
    # Top 6 errors 
    most_important_errors = sorted_dela_errors[-6:]
    display_errors(most_important_errors, X_test_errors, Y_cls_errors, Y_true_errors,
                   X_test.shape[1], X_test.shape[2])
    print("Biggest error probabilities are: ", Y_pred_errors_prob[most_important_errors])
    
    ### Plot predictions
    predicted = Y_cls[:15]
    fig, axs = plt.subplots(nrows=3, ncols=5, figsize=(12, 6))

    for i, ax in enumerate(axs.flat):
        ax.imshow(X_test[i].reshape(img_rows, img_cols, 3),
                  interpolation='nearest')
        ax.text(0, 0, flowers[predicted[i]], 
                color='black', bbox=dict(facecolor='white', alpha=1))
        ax.axis('off')

# Training

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.1764, random_state=42, stratify=y_Train) # 15% validation set
X_train.shape

(2296, 200, 200, 3)

In [ ]:
print("Für die einzelnen Klassen sind so viele Instanzen im Validierungsdatensatz:\n", 
      flowers, "\n", np.sum(y_val, axis=0))

Für die einzelnen Klassen sind so viele Instanzen im Validierungsdatensatz:
 ['ButterflyBush', 'GrapeHyacinth', 'BalloonFlower', 'BleedingHeart', 'FrangipaniFlower', 'Daisy', 'Black-eyedSusan', 'BlanketFlower', 'Waterlilies'] 
 [61. 55. 65. 62. 48. 42. 61. 49. 49.]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

nb_epoch = 40
batch_size = 128

# number of convolutional filters to use
nb_filters = 64
# number of nodes for dense layers
dense_nodes = 128
# dropout rate
rate_do = 0.3 

# convolution kernel size
nb_conv = 4
#size of pooling area for max pooling
nb_pool = 2


# Manually trying some structures leads to something like this working quite well 
# (2 dense layers does improve the result)
layers_base = [
    Conv2D(nb_filters, kernel_size=nb_conv, padding='valid', activation='relu', input_shape=shape_ord),
    MaxPooling2D(pool_size=nb_pool),
    Conv2D(nb_filters / 2, kernel_size=nb_conv, padding='valid', activation='relu'),
    MaxPooling2D(pool_size=nb_pool),
    Conv2D(nb_filters / 4, kernel_size=nb_conv, padding='valid', activation='relu'),
    MaxPooling2D(pool_size=nb_pool),
    Flatten(),
    Dropout(rate_do),
    Dense(dense_nodes, activation='relu'),
    Dropout(rate_do),
    Dense(dense_nodes, activation='relu'),
    Dropout(rate_do),
    Dense(len(flowers), activation='softmax')
]


model = Sequential(layers_base)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])          
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 197, 197, 64)      3136      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 95, 95, 32)        32800     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 16)        8208      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 22, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 7744)              0

In [ ]:
from tensorflow.keras.callbacks import Callback

class HistoryEpoch(Callback):
    def __init__(self, data):
        self.data = data        
        
    def on_train_begin(self, logs={}):
        self.loss = []
        self.acc = []

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.data
        l, a = self.model.evaluate(x, y, verbose=0)
        self.loss.append(l)
        self.acc.append(a)


train_hist = HistoryEpoch((X_train, y_train))
val_hist = HistoryEpoch((X_val, y_val))

hist = model.fit(X_train, y_train, batch_size = batch_size, 
                 epochs = nb_epoch, verbose=1, 
                 validation_data = (X_val, y_val),
                 callbacks = [val_hist, train_hist])

NameError: ignored

In [ ]:
plot_history(hist)

# Hyperparametertuning mit Gridsearch

In [ ]:
import itertools

def build_model_from_params(params):
    layers = [
        Conv2D(params["num_filters"], kernel_size=nb_conv, padding='valid', activation='relu', input_shape=shape_ord),
        MaxPooling2D(pool_size=nb_pool),
        Conv2D(params["num_filters"] / 2, kernel_size=nb_conv, padding='valid', activation='relu'),
        MaxPooling2D(pool_size=nb_pool),
        Conv2D(params["num_filters"] / 4, kernel_size=nb_conv, padding='valid', activation='relu'),
        MaxPooling2D(pool_size=nb_pool),
        Flatten(),
        Dropout(params["dropout"]),
        Dense(params["dense_nodes"], activation='relu'),
        Dropout(params["dropout"]),
        Dense(params["dense_nodes"], activation='relu'),
        Dropout(params["dropout"]),
        Dense(len(flowers), activation='softmax')
    ]
  
    model = Sequential(layers)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model 


param_space = {
    'batch_size': [128, 256],
    'num_filters': [32, 64],
    'dropout': [0.3, 0.4, 0.5],
    'dense_nodes': [64, 128]
}

value_combis = itertools.product(*[v for v in param_space.values()])

param_combis = [{key:value for key, value in zip(param_space.keys(), combi)} for combi in value_combis]

print(f"Es werden {len(param_combis)} Parameterkombination getestet:")

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

search_results = []

for idx, params in enumerate(param_combis):
    print(f"Starte Run {idx+1}/{len(param_combis)} mit den Parameteren: {params}")

    string_config = ""
    for key, value in params.items():
      string_config += "_" + key + "=" + str(value)

    # save best model according to validation accuracy
    filepath = f"paramsearch{string_config}.hdf5"
    checkpoint = ModelCheckpoint(
        filepath, monitor="val_accuracy", verbose=0, save_best_only=True, mode="max"
    )

    model = build_model_from_params(params)
    
    # train the model
    fit_results = model.fit(
        x = X_train, y = y_train, 
        batch_size = params["batch_size"], epochs = nb_epoch, 
        validation_data = (X_val, y_val), 
        callbacks = [checkpoint],
        verbose = 0
    )

    # extract the best validation scores
    best_val_epoch    = np.argmax(fit_results.history['val_accuracy'])
    best_val_acc      = np.max(fit_results.history['val_accuracy'])
    best_val_acc_loss = fit_results.history['val_loss'][best_val_epoch]

    # get correct training accuracy
    best_model = load_model(filepath)
    best_val_acc_train_loss, best_val_acc_train_acc = best_model.evaluate(X_train, y_train)

    # store results
    search_results.append({
        **params,
        'best_val_acc': best_val_acc,
        'best_val_loss': best_val_acc_loss,
        'best_train_acc': best_val_acc_train_acc,
        'best_train_loss': best_val_acc_train_loss,
        'best_val_epoch': best_val_epoch
    })

In [ ]:
resultsDF = pd.DataFrame(search_results)
resultsDF.sort_values('best_val_acc', ascending=False)